In [4]:
import pandas as pd
from pathlib import Path
from costumDataset import *
data_path = Path(r'..\Data')
list(data_path.glob('*'))

[WindowsPath('../Data/aug_data_labels.csv'),
 WindowsPath('../Data/data_labels.csv'),
 WindowsPath('../Data/data_labels.txt'),
 WindowsPath('../Data/fixed_data'),
 WindowsPath('../Data/without_rips'),
 WindowsPath('../Data/without_rips-20220614T153319Z-001.zip'),
 WindowsPath('../Data/with_rips'),
 WindowsPath('../Data/with_rips-20220614T153300Z-001.zip'),
 WindowsPath('../Data/with_rips-20220614T153300Z-002.zip')]

In [5]:
%load_ext autoreload
%autoreload 2

In [3]:
csv_path = data_path/'aug_data_labels.csv'
csv_df = pd.read_csv(csv_path)
csv_df.drop_duplicates(subset=['Name'], inplace=True)
csv_df.to_csv(csv_path, index=False)

In [4]:
#Thanks to: https://stackoverflow.com/questions/72552647/pytorch-fasterrcnn-gives-no-output
from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights, FasterRCNN_ResNet50_FPN_V2_Weights
import torchvision

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)

In [6]:
import pandas as pd
from pathlib import Path
from costumDataset import *
data_path = Path(r'..\Data')
from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights, FasterRCNN_ResNet50_FPN_V2_Weights
import torchvision

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT,
                                                            progress = True, 
                                                             num_classes = 2,
                                                             trainable_backbone_layers =2,
                                                           
                                                            )
df = pd.read_csv(data_path/'aug_data_labels.csv')
df = df[df['label'] == 1]

img_dir = str(data_path/'fixed_data') + '\\'
trans = T.ToTensor()

train_ds = RipCurrentDataset(df, img_dir, trans)

train_dl = DataLoader(train_ds, batch_size=16, shuffle=True)


for images, targets in train_dl:

    
    boxes = targets['box']
    labels = targets['labels']
    new_targets = []
    for box, label in zip(boxes, labels):
        tmp_d = {'boxes': box, 'labels': label}
        new_targets.append(tmp_d)
        assert tmp_d['boxes'].shape[0] == tmp_d['labels'].shape[0]
        
    assert len(images) == len(new_targets)
    
    output = model(images, new_targets)
    model.eval()
    
    break



ValueError: The parameter '2' expected value 91 but got 2 instead.

In [2]:
predictions = model(images)


In [22]:
# predictions

import torchvision.transforms as T
from PIL import Image, ImageDraw


new_targets[0]


img = T.ToPILImage()(images[0].squeeze_(0))

# draw_rect(img, new_targets[0]['boxes'])
for box in predictions[0]['boxes']:
    draw_rect(img, [box])


[{'boxes': tensor([[102.8281, 168.2507, 198.2979, 293.8721],
          [ 36.2444,   3.3176,  72.7672,  73.2350],
          [ 23.4281,   6.8971, 297.1317, 290.9916],
          [ 15.3913,   0.0000, 292.0162, 289.3502],
          [ 49.8298,  13.6729, 286.2994, 298.1857]], grad_fn=<StackBackward0>),
  'labels': tensor([16, 16, 23, 16, 17]),
  'scores': tensor([0.2372, 0.2080, 0.0798, 0.0746, 0.0616], grad_fn=<IndexBackward0>)},
 {'boxes': tensor([[ 98.4122, 225.0630, 215.7074, 298.8628],
          [ 69.7705, 207.5590,  94.2809, 223.1382],
          [ 70.0334, 207.4812,  94.2688, 222.8797],
          [117.6527,   6.9166, 283.9580, 132.7610],
          [ 70.6363, 208.0485,  94.5545, 223.0778]], grad_fn=<StackBackward0>),
  'labels': tensor([64, 18, 16, 28, 21]),
  'scores': tensor([0.3104, 0.2635, 0.1137, 0.0763, 0.0703], grad_fn=<IndexBackward0>)},
 {'boxes': tensor([], size=(0, 4), grad_fn=<StackBackward0>),
  'labels': tensor([], dtype=torch.int64),
  'scores': tensor([], grad_fn=<IndexBa

In [5]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)


# For training


images, boxes = torch.rand(4, 3, 600, 1200), torch.rand(4, 11, 4)
boxes[:, :, 2:4] = boxes[:, :, 0:2] + boxes[:, :, 2:4]
labels = torch.randint(1, 91, (4, 11))
images = list(image for image in images)
targets = []
for i in range(len(images)):
    d = {}
    d['boxes'] = boxes[i]
    d['labels'] = labels[i]
    targets.append(d)
    
    
output = model(images, targets)


# For inference
model.eval()


x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]

predictions = model(x)



In [6]:
d

{'boxes': tensor([[0.0363, 0.7126, 0.5770, 0.9409],
         [0.7278, 0.8902, 1.0662, 1.0080],
         [0.0739, 0.9098, 1.0553, 1.1821],
         [0.0848, 0.1756, 1.0565, 0.9715],
         [0.4173, 0.8681, 0.8439, 1.0397],
         [0.8167, 0.1354, 1.6879, 0.8188],
         [0.8472, 0.3822, 1.6970, 1.2252],
         [0.8038, 0.3779, 0.9716, 0.6015],
         [0.5040, 0.4569, 1.3133, 1.1090],
         [0.7390, 0.3664, 1.1639, 1.3458],
         [0.4682, 0.4418, 0.8152, 0.6511]]),
 'labels': tensor([72,  6,  1, 90,  1, 71, 27, 73, 78,  6, 59])}

In [10]:
len(images)


4

In [ ]:
df = pd.read_csv(data_path/'aug_data_labels.csv')
df = df[df['label'] == 1]

img_dir = str(data_path/'fixed_data') + '\\'
trans = T.ToTensor()

train_ds = RipCurrentDataset(df, img_dir, trans)

train_dl = DataLoader(train_ds, batch_size=16, shuffle=True)

a = next(iter(train_dl))

In [ ]:
import pandas as pd

from costumDataset import *
from resize_and_augmentation import *
from pathlib import Path

data_path = Path(r'..\Data')
S = np.array((300, 300))
data_labels_path = str(data_path/'data_labels.csv')
im_path = str(data_path/'with_rips') + '\\'
target_path = str(data_path/'fixed_data') + '\\'
fix_size_and_aug(data_labels_path, S, im_path, target_path)